In [49]:
import json
import pickle
import pandas as pd

In [50]:
data_dir = './test_data/'

In [51]:
def grab_children(dest, collect):
    if hasattr(dest, 'children'):
        for child in dest.children:
            collect.append(child.__dict__)
            grab_children(child, collect)
    return collect

In [52]:
def climb_from_dest(dest):
    kidnapped = grab_children(dest, collect = []) # kidnapping jokes are not okay
    climb = pd.DataFrame.from_dict(kidnapped) # neither are dict jokes
    if max(climb.shape) > 0:
        climb.set_index(climb.href.values, inplace = True, verify_integrity = False)
        return climb

In [53]:
def combine_pickle(data_dir):
    import os
    
    collect_climb = []
    for area in os.listdir(data_dir):
        pkl = '.pickle'
        if area[-len(pkl):] != pkl:
            pass
        else:
            dest = pickle.load(open(data_dir + area, 'rb'))        
            partial = climb_from_dest(dest)
            collect_climb.append(partial)
            print len(collect_climb), area[:-len(pkl)]
    
    climb = pd.concat(collect_climb)
    return climb

In [54]:
climb = combine_pickle(data_dir)

1 boxcarwoodyboulders
2 hillcrestboulders
3 woodcrestboulders


In [55]:
climb.shape

(103, 22)

In [63]:
climb.columns

Index([u'area_hierarchy', u'boulder', u'children', u'children_href',
       u'commitment', u'description', u'fa', u'feet', u'getting_there',
       u'grade', u'href', u'is_area', u'is_route', u'location', u'name',
       u'nickname', u'page_views', u'pitches', u'protect_rate', u'protection',
       u'star_rating', u'type'],
      dtype='object')

In [64]:
len(climb['href'].unique()) / float(len(climb['href']))
# YAY the hrefs are unique!!

1.0

In [65]:
def combine_text(jess):
    
    # ordered by what makes sense
    text_field = ['getting_there', 'description', 'protection']
    
    # loop through text fields appending to a list
    # TODO do this with a map
    txt_collect = []
    for txt in text_field:
        if not pd.isnull(jess[txt]):
            txt_collect.append(jess[txt])
    
    return "\n".join(txt_collect)

In [66]:
def get_total_climb_description(climb):
    collect = []
    for href, cmb in climb.iterrows():
        cmbtxt = combine_text(cmb)
        collect.append(cmbtxt)

    print "Combined text from %d rows" % len(collect)
    return collect

In [67]:
import sys
import numpy as np

In [68]:
pct_area = np.mean(climb['is_area'])
pct_route = np.mean(climb['is_route'])
if pct_area + pct_route < (1 - sys.float_info.epsilon):
    # TODO identify records with (route nor area) logical
    print "One or more records WAS NEITHER ROUTE NOR AREA"

In [69]:
descriptive = get_total_climb_description(climb)
# print "The len of descriptive is %d" % len(descriptive['cmb_txt'])
# descriptive['cmb_txt'][-3:] # look at a few

Combined text from 103 rows


In [70]:
from sklearn.feature_extraction.text import CountVectorizer


CV12 = CountVectorizer(decode_error='ignore', stop_words='english', ngram_range=(1, 2))
# also consider max_df=0.5, min_df=1
word_count = CV12.fit_transform(descriptive)

In [71]:
word_count.shape

(103, 3683)